<a href="https://colab.research.google.com/github/kirtiman24/Emoji_Predictor/blob/master/Emoji_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd


**Installing the emoji package**

In [6]:
!pip install emoji

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=e9adda3349e74b7e0ccf0f74481701195089536e2b7f5987c31d9271e12c0355
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import emoji as emoji

In [0]:
train=pd.read_csv('/content/ET 2.csv',header=None)

test=pd.read_csv('/content/ETEST .csv',header=None)


In [9]:
train.head(5)

,0,1
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [10]:
test.head(5)

,0,1
0,I want to eat,4
1,he did not answer,3
2,he got a raise,2
3,she got me a present,0
4,ha ha ha it was so funny,2


In [0]:
emoji_dictionary = {"0": "\u2764\uFE0F",   
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }


In [12]:
emoji.emojize(":beaming_face_with_smiling_eyes:")

'😁'

In [13]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [14]:
data = train.values
for i in range(5):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴


In [0]:
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

**Preparing the target variable by converting it into the categorical variable**

In [55]:
y_train=to_categorical(train[1][:100])
y_cv=to_categorical(train[1][100:])
y_test=to_categorical(test[1])
print(y_train.shape)
print(y_cv.shape)
print(y_test.shape)

(100, 5)
(32, 5)
(56, 5)


**Preparing input data to be fed into the LSTM RNN layer.**

In [0]:
t = Tokenizer()
t.fit_on_texts(train[0])
vocab_size = len(t.word_index) + 1
encoded = t.texts_to_sequences(train[0])

In [31]:
#Carrying out padding of the input data to preserve the integrity of the original data 
padded = pad_sequences(encoded, maxlen=25, padding='post')
print(padded[:5])

[[83 84  9 15 52  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [ 1  6 53 28 18 85  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [16  3  4 86 54 55 10 34  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [35  2  7 87  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [22  3 34  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]]


In [53]:
x_train_padded=padded[:100]
x_cv=padded[100:]
x_train_padded.shape
x_cv.shape


(32, 25)

**Using pre-trained glove embeddings containig 50 dimensional matrix.**

In [0]:
embeddings_index = dict()
f = open('glove.6B.50d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

**Creating the LSTM layer architecture**

In [0]:
from keras.layers import Embedding, Flatten, Dense, LSTM, Dropout
from keras.models import Sequential

In [0]:
from keras.callbacks import ModelCheckpoint
filepath="model.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
model = Sequential()
e = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length = 25, trainable=True)
model.add(e)
model.add(LSTM(128,activation = 'tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128,activation = 'tanh', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(128,activation = 'tanh', return_sequences=False))
model.add(Dropout(0.4))
model.add(Dense(5, activation = 'softmax'))

In [73]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 25, 50)            13050     
_________________________________________________________________
lstm_10 (LSTM)               (None, 25, 128)           91648     
_________________________________________________________________
dropout_10 (Dropout)         (None, 25, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 25, 128)           131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 25, 128)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)              

In [0]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

In [75]:
model.fit(x_train_padded, y_train, epochs = 100, batch_size = 512,callbacks = callbacks_list,validation_data=(x_cv,y_cv))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 100 samples, validate on 32 samples
Epoch 1/100
100/100 [==============================] - 1s 14ms/step - loss: 1.6067 - acc: 0.2300 - val_loss: 1.5979 - val_acc: 0.2188
Epoch 2/100
100/100 [==============================] - 0s 3ms/step - loss: 1.5807 - acc: 0.2800 - val_loss: 1.5981 - val_acc: 0.2188
Epoch 3/100
100/100 [==============================] - 0s 3ms/step - loss: 1.5516 - acc: 0.3100 - val_loss: 1.6292 - val_acc: 0.2188
Epoch 4/100
100/100 [==============================] - 0s 3ms/step - loss: 1.5229 - acc: 0.2900 - val_loss: 1.6946 - val_acc: 0.2188
Epoch 5/100
100/100 [==============================] - 0s 3ms/step - loss: 1.5347 - acc: 0.3300 - val_loss: 1.6660 - val_acc: 0.3125
Epoch 6/100
100/100 [==============================] - 0s 3ms/step - loss: 1.4807 - acc: 0.3900 - val_loss: 1.6235 - val_acc: 0.3438
Epoch 7/100
100/100 [==============================] - 0s 3ms/step - loss: 1.4640 - acc: 0.3800 - val_loss: 1.5745 - val_acc: 0.2812
Epoch 8/100
100/100 [==

**Preparing test data**

In [0]:
t = Tokenizer()
t.fit_on_texts(test[0])
vocab_size = len(t.word_index) + 1
encoded = t.texts_to_sequences(test[0])

In [77]:
padded = pad_sequences(encoded, maxlen=25, padding='post')
print(padded[:2])

[[ 1 23  9 46  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [10 11 12 47  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]]


In [79]:
x_test_padded=padded[:]
print(x_test_padded.shape)

(56, 25)


In [80]:
model.evaluate(x_test_padded,y_test)

56/56 [==============================] - 0s 1ms/step


[6.52942909513201, 0.1964285671710968]

In [0]:
pred = model.predict_classes(x_test_padded)

**Predicted emoji matched with actual emoji**

In [87]:
for i in range(10):
    print(' '.join(train[0][i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(y_test[i]))])) #Printing the actual emoji
    print(emoji.emojize(emoji_dictionary[str(pred[i])])) # Printing the predicted emoji


never talk to me again
🍴
😁
I am proud of your achievements
😓
😓
I t   i s   t h e   w o r s t   d a y   i n   m y   l i f e
😁
❤️
M i s s   y o u   s o   m u c h
❤️
😁
f o o d   i s   l i f e
😁
😁
I   l o v e   y o u   m u m
❤️
😓
S t o p   s a y i n g   b u l l s h i t
❤️
😓
c o n g r a t u l a t i o n s   o n   y o u r   a c c e p t a n c e
❤️
😁
T h e   a s s i g n m e n t   i s   t o o   l o n g
🍴
❤️
I   w a n t   t o   g o   p l a y
😁
😓
